In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import dgl
import dgl.data
import dgl.nn as gnn

from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

Using backend: pytorch


# Постановка задачи и ход решения

Задача: предсказать какую-нибудь характеристику на уровне целого графа

Типичный датасет:
* содержит некоторое кол-во графов
* у каждого графа есть фичи на узлах и метка на весь граф 

Для примера используем синтетический датасет GIN. 

План:
1. Подготовить батч графов
2. Прогнать батч через сеть и получить представления узлов в батче
3. Получить представление графов в батче
4. Классификация графов в батча






## Разбиение датасета на батчи

Проблема: нужно уметь разбивать датасет на мини-батчи. 

Объединенный граф (батч-граф) состоит из нескольких графов, объединенных в один в виде несвязных компонент. Фичи узлов и ребер сохраняются. Это такой же граф, как и обычный `DGLGraph`, но содержит доп. информацию для восстановления исходных графов. Развернуть графы назад можно с помощью метода `unbatch`.

Примечание: большинство операций на батч-графом сотрут информацию о структуре батча.

![](./assets/img/07_dgl_graph_classification_batch.png)

In [7]:
G1 = dgl.graph((torch.tensor([0, 1, 2]), torch.tensor([1, 2, 3])))
G2 = dgl.graph((torch.tensor([0, 0, 0, 1]), torch.tensor([0, 1, 2, 0])))

batch = dgl.batch([G1, G2])
print(f'{batch=}')
print(f'{batch.batch_size=}')
print(f'{batch.batch_num_nodes()=}')
print(f'{batch.batch_num_edges()=}')

batch=Graph(num_nodes=7, num_edges=7,
      ndata_schemes={}
      edata_schemes={})
batch.batch_size=2
batch.batch_num_nodes()=tensor([4, 3])
batch.batch_num_edges()=tensor([3, 4])


## Readout

Workflow:
1. Создаем как обычно двухслойную сеть. На вход придет батч-граф. 
2. Дальше нужно сагрегировать представления узлов (и возможно ребер) чтобы получить представление графа в целом (самый простой вариант - усреднить с помощью `dgl.mean_nodes()` или суммировать с помощью `dgl.readout_nodes()`). Этот процесс называют `readout`. `DGL` предоставляет набор функций, которые могут работать с батч-графами и получать для них представление. 

![](./assets/img/07_dgl_graph_classification_graph_classifier.png)

In [14]:
G1 = dgl.graph(([0, 1], [1, 0]))
G1.ndata['h'] = torch.tensor([[1, 2], [3, 4]]).float()
G2 = dgl.graph(([0, 1], [1, 2]))
G2.ndata['h'] = torch.tensor([[5, 6], [7, 8], [9, 10]]).float()

batch = dgl.batch([G1, G2])

print(f'{dgl.readout_nodes(G1, "h")=}')
print(f'{dgl.readout_nodes(G2, "h")=}')
print(f'{dgl.readout_nodes(batch, "h")=}')

dgl.readout_nodes(G1, "h")=tensor([[4., 6.]])
dgl.readout_nodes(G2, "h")=tensor([[21., 24.]])
dgl.readout_nodes(batch, "h")=tensor([[ 4.,  6.],
        [21., 24.]])


## Разбиение датасета на батчи с помощью `GraphDataLoader`

Что разбить датасет из графов на батчи, используем `GraphDataLoader`, который работает со стандартными сэмплерами из `torch`.

Каждый элемент в датасете представляет собой пару (граф, метка). `GraphDataLoader` при итерации по нему возвращает два объекта: объедененный граф для батча и вектор с метками для каждого графа из батча. 


In [ ]:
dataset = dgl.data.GINDataset('PROTEINS', self_loop=True)
print('Node feature dimensionality:', dataset.dim_nfeats)
print('Number of graph categories:', dataset.gclasses)

In [26]:
num_examples = len(dataset)
num_train = int(num_examples * .8)

train_sampler = SubsetRandomSampler(torch.arange(num_train))
test_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))

train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=5, drop_last=False)
test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=5, drop_last=False)

In [ ]:
it = iter(train_dataloader)
batched_graph, labels = next(it)
print('Кол-во узлов в каждом графе из батча:', batched_graph.batch_num_nodes())
print('Кол-во ребер в каждом графе из батча:', batched_graph.batch_num_edges())

In [ ]:
# Получить исходные графы из минибатча
graphs = dgl.unbatch(batched_graph)
print(len(graphs))

In [24]:
class GCN(nn.Module):
    def __init__(self, n_inputs, n_hidden, num_classes):
        super().__init__()
        self.conv1 = gnn.GraphConv(n_inputs, n_hidden)
        self.conv2 = gnn.GraphConv(n_hidden, n_hidden)
        self.linear = nn.Linear(n_hidden, num_classes)

    def forward(self, G, features):
        out = F.relu(self.conv1(G, features))
        out = F.relu(self.conv2(G, out))
        with G.local_scope():
            G.ndata['h'] = out
            out = dgl.mean_nodes(G, 'h')
            out = self.linear(out)
        return out


In [27]:
n_inputs, n_hidden, n_out = dataset.dim_nfeats, 16, dataset.gclasses
model = GCN(n_inputs, n_hidden, n_out)
optimizer = torch.optim.Adam(model.parameters(), lr=.01)

n_epochs = 20
for epoch in range(n_epochs):
    for batched_graph, labels in train_dataloader:
        pred = model(batched_graph, batched_graph.ndata['attr'].float())
        loss = F.cross_entropy(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

num_correct = 0
num_tests = 0
for batched_graph, labels in test_dataloader:
    pred = model(batched_graph, batched_graph.ndata['attr'].float())
    num_correct += (pred.argmax(1) == labels).sum().item()
    num_tests += len(labels)

print('Test accuracy:', num_correct / num_tests)


Test accuracy: 0.32286995515695066


Для случая гетерографов нужно:
1. Использовать модели, работающие с гетерографами
2. Изменить readout: сначала, например, усредняем по типам узлов; потом - суммируем средние.

```
hg = 0
for ntype in g.ntypes:
    hg = hg + dgl.mean_nodes(g, 'h', ntype=ntype)
```